<a href="https://colab.research.google.com/github/jsaraujo5081/CursoPythonConfenalco/blob/notebooks/04AnalisisDatos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ANÁLISIS DE DATOS EN PYTHON

## Contenido

* Importar data frame
* Ajuste preliminar de variables
* Análisis estadístico y agregación
* Ingeniería de variables

## Importar data frame 

In [2]:
#Cargar librerias para análisis de datos
import pandas as pd
import numpy as np

In [26]:
#Importar data frame
filepath = "https://raw.githubusercontent.com/jsaraujo5081/CursoPythonConfenalco/data/Data_Matriculacion.csv"
df_autos = pd.read_table(filepath, header = 0, sep = ",")

In [4]:
df_autos.head(5)

,Cod_Vehiculo,Periodo,Mes,Canton,Provincia,Marca,Modelo,Pais_Origen,Clase,Sub_Clase,Cilindraje,Combustible,Servicio,Valor_Avaluo
0,5396696,2018,3,GUAYAQUIL,GUAYAS,CHEVROLET,FVR 34K AC 7.8 2P 4X2 TM DIESEL CN,JAPON,CAMION,CAMION,7800,DIESEL,PARTICULAR,49494.00
1,5408757,2018,2,DURAN,GUAYAS,CHEVROLET,FVZ 34P AC 7.8 2P 6X4 TM DIESEL CN,JAPON,CAMION,CAMION,7800,DIESEL,PARTICULAR,58794.00
2,5421129,2018,1,QUITO,PICHINCHA,JAC,HFC1035KD 2.8 2P 4X2 TM DIESEL,CHINA POPULAR,CAMION,CAMION,2800,DIESEL,PARTICULAR,14904.00
3,5439441,2018,2,LOJA,LOJA,CHEVROLET,NPR 75H 5.2 2P 4X2 TM DIESEL,JAPON,CAMION,FURGON-C,5200,DIESEL,PARTICULAR,41678.46
4,5439482,2018,3,GUAYAQUIL,GUAYAS,CHEVROLET,NQR 75L 5.2 2P 4X2 TM DIESEL CN,JAPON,CAMION,CAMION,5200,DIESEL,PARTICULAR,25674.00


In [5]:
#Estructura del data frame
df_autos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 349109 entries, 0 to 349108
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Cod_Vehiculo  349109 non-null  int64  
 1   Periodo       349109 non-null  int64  
 2   Mes           349109 non-null  int64  
 3   Canton        349109 non-null  object 
 4   Provincia     349109 non-null  object 
 5   Marca         349109 non-null  object 
 6   Modelo        349109 non-null  object 
 7   Pais_Origen   349109 non-null  object 
 8   Clase         349109 non-null  object 
 9   Sub_Clase     349109 non-null  object 
 10  Cilindraje    349109 non-null  int64  
 11  Combustible   349109 non-null  object 
 12  Servicio      349109 non-null  object 
 13  Valor_Avaluo  349100 non-null  float64
dtypes: float64(1), int64(4), object(9)
memory usage: 37.3+ MB


## Ajuste preliminar de variables

In [27]:
#Cambiar tipos de variables (Forma 1)
df_autos["Cod_Vehiculo"] = str(df_autos["Cod_Vehiculo"])

In [28]:
#Forma 2
df_autos["Canton"] = df_autos["Canton"].astype("category")
df_autos["Provincia"] = df_autos["Provincia"].astype("category")
df_autos["Marca"] = df_autos["Marca"].astype("category")
df_autos["Clase"] = df_autos["Clase"].astype("category")
df_autos["Sub_Clase"] = df_autos["Sub_Clase"].astype("category")
df_autos["Combustible"] = df_autos["Combustible"].astype("category")
df_autos["Servicio"] = df_autos["Servicio"].astype("category")

In [29]:
#Estructura del data frame
df_autos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 349109 entries, 0 to 349108
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype   
---  ------        --------------   -----   
 0   Cod_Vehiculo  349109 non-null  object  
 1   Periodo       349109 non-null  int64   
 2   Mes           349109 non-null  int64   
 3   Canton        349109 non-null  category
 4   Provincia     349109 non-null  category
 5   Marca         349109 non-null  category
 6   Modelo        349109 non-null  object  
 7   Pais_Origen   349109 non-null  object  
 8   Clase         349109 non-null  category
 9   Sub_Clase     349109 non-null  category
 10  Cilindraje    349109 non-null  int64   
 11  Combustible   349109 non-null  category
 12  Servicio      349109 non-null  category
 13  Valor_Avaluo  349100 non-null  float64 
dtypes: category(7), float64(1), int64(3), object(3)
memory usage: 21.7+ MB


## Analisis estadístico y agregación

In [79]:
# Resumen estadístico (Variables numéricas)
df_autos.describe(include = np.number)

,Periodo,Mes,Cilindraje,Valor_Avaluo
count,349109.000000,349109.000000,349109.000000,3.491000e+05
mean,2018.844814,6.590818,2105.605126,2.871484e+04
std,0.773847,3.509395,1531.513488,2.289490e+04
min,2018.000000,1.000000,0.000000,4.312200e+02
25%,2018.000000,3.000000,1500.000000,1.799000e+04
50%,2019.000000,7.000000,1600.000000,2.279200e+04
75%,2019.000000,10.000000,2200.000000,2.999000e+04
max,2020.000000,12.000000,16000.000000,1.530000e+06


In [80]:
#Correlacion entre Cilindraje y Avaluo
df_cilaval = df_autos[["Cilindraje","Valor_Avaluo"]]
df_cilaval.corr()

,Cilindraje,Valor_Avaluo
Cilindraje,1.000000,0.753037
Valor_Avaluo,0.753037,1.000000


In [81]:
#Resumen estadístico (Variables no numéricas)
df_autos.describe(include = [object,"category"])

,Cod_Vehiculo,Canton,Provincia,Marca,Modelo,Pais_Origen,Clase,Sub_Clase,Combustible,Servicio
count,349109,349109,349109,349109,349109,349109,349109,349109,349109,349109
unique,1,221,24,147,3352,32,9,49,7,2
top,0 5396696\n1 5408757\n2 ...,QUITO,PICHINCHA,CHEVROLET,SAIL LS AC 1.5 4P 4X2 TM,ECUADOR,AUTOMOVIL,STATION WAGON,GASOLINA,PARTICULAR
freq,349109,125233,137207,93321,11752,78498,140308,152053,274795,342023


In [82]:
#Serie con conteo por cantón
frec_canton = df_autos["Canton"].value_counts()

#Transformar serie a data frame creando primero un diccionario
d_frec_canton = {"Canton":frec_canton.index,"Frecuencia":frec_canton}
df_frec_canton = pd.DataFrame(d_frec_canton)
df_frec_canton.index = range(0,len(frec_canton))

#Incorporar frecuencia relativa
df_frec_canton["Frec_Relativa"] = np.round(100*df_frec_canton.Frecuencia/np.sum(df_frec_canton.Frecuencia),2)

#Ver resultado
df_frec_canton

,Canton,Frecuencia,Frec_Relativa
0,QUITO,125233,35.87
1,GUAYAQUIL,87990,25.20
2,CUENCA,16723,4.79
3,AMBATO,16476,4.72
4,RIOBAMBA,8048,2.31
...,...,...,...
216,TAISHA,4,0.00
217,SOZORANGA,3,0.00
218,ALFREDO BAQUERIZO MORENO (JUJAN),2,0.00
219,CHILLA,2,0.00


In [83]:
#Conteos o tablas de frecuencia por provincia
frec_provincia = df_autos["Provincia"].value_counts()
d_frec_provincia = {"Provincia":frec_provincia.index,"Frecuencia":frec_provincia}
df_frec_provincia = pd.DataFrame(d_frec_provincia)
df_frec_provincia.index = range(0,len(frec_provincia))
df_frec_provincia["Frec_Relativa"] = np.round(100*df_frec_provincia.Frecuencia/np.sum(df_frec_provincia.Frecuencia),2)
df_frec_provincia

,Provincia,Frecuencia,Frec_Relativa
0,PICHINCHA,137207,39.30
1,GUAYAS,95730,27.42
2,AZUAY,24597,7.05
3,TUNGURAHUA,18937,5.42
4,MANABI,14788,4.24
5,IMBABURA,9456,2.71
6,CHIMBORAZO,8667,2.48
7,LOJA,7249,2.08
8,EL ORO,6795,1.95
9,COTOPAXI,4416,1.26


In [84]:
#Otras tablas de frecuencia que desee para variables categóricas

In [85]:
#Obtener el avalúo promedio por año y mes
df_autos.groupby(["Periodo","Mes"], as_index=False)["Valor_Avaluo"].mean()

,Periodo,Mes,Valor_Avaluo
0,2018,1,28831.930785
1,2018,2,28679.311236
2,2018,3,27965.582825
3,2018,4,28419.335980
4,2018,5,28971.141936
5,2018,6,28252.722785
6,2018,7,28810.357741
7,2018,8,28959.815692
8,2018,9,29280.635670
9,2018,10,30817.165259


In [86]:
#Obtener para cada periodo y mes el promedio del avaluo y el conteo de vehículos
df_autos.groupby(["Periodo","Mes"], as_index=False).agg({"Valor_Avaluo":"mean","Cod_Vehiculo":"count"})

,Periodo,Mes,Valor_Avaluo,Cod_Vehiculo
0,2018,1,28831.930785,11379
1,2018,2,28679.311236,9394
2,2018,3,27965.582825,11841
3,2018,4,28419.335980,11142
4,2018,5,28971.141936,11682
5,2018,6,28252.722785,12378
6,2018,7,28810.357741,11439
7,2018,8,28959.815692,11451
8,2018,9,29280.635670,10942
9,2018,10,30817.165259,11543


In [87]:
#Crear tabla pivote entre paises y clase que cuente casos
df_pais_clase = df_autos.pivot_table("Cod_Vehiculo", index = "Pais_Origen", columns = "Clase", aggfunc = "count")
df_pais_clase

Clase,AUTOMOVIL,CAMION,CAMIONETA,ESPECIAL,JEEP,OMNIBUS,TANQUERO,TRAILER,VOLQUETA
Pais_Origen,,,,,,,,,
ALEMANIA,1162,33,80,61,966,87,0,281,25
ARGENTINA,4,0,5176,40,1228,0,0,0,0
AUSTRALIA,0,0,2,0,0,0,0,0,0
BELGICA,7,0,0,0,160,0,0,0,0
BRASIL,2621,45,57,0,2928,1046,0,28,14
CANADA,33,0,2,0,750,0,0,0,0
CHINA POPULAR,12248,3595,16688,97,28794,321,24,800,87
COLOMBIA,35572,7120,0,223,2128,453,99,86,227
COREA DEL SUR,19914,661,978,0,11921,29,0,16,50


In [88]:
#Construir tabla pivote por clase y combustible pero con el promedio de avalúo
df_clase_comb = df_autos.pivot_table("Valor_Avaluo", index = "Clase", columns = "Combustible", aggfunc = "mean")
df_clase_comb

Combustible,ALCOHOL,DIESEL,DUAL_GAS_GASOLINA,ELECTRICO,GAS,GASOLINA,HIBRIDO
Clase,,,,,,,
AUTOMOVIL,16056.666667,21251.031506,19890.0,24536.565109,NaN,18162.056685,28341.060511
CAMION,NaN,44942.701923,NaN,6848.120000,NaN,13490.293839,NaN
CAMIONETA,19990.000000,33099.407690,NaN,9600.048833,NaN,26511.261175,73990.750000
ESPECIAL,NaN,151110.994815,NaN,NaN,NaN,189399.801538,NaN
JEEP,49990.000000,51277.059768,NaN,13555.010198,40916.11,31168.051475,33940.630404
OMNIBUS,NaN,100306.808271,NaN,421160.000000,NaN,245000.000000,NaN
TANQUERO,NaN,71864.232822,NaN,NaN,NaN,NaN,NaN
TRAILER,NaN,117399.307413,148000.0,NaN,NaN,101400.000000,NaN
VOLQUETA,NaN,112400.748196,NaN,NaN,NaN,NaN,NaN


## Ingeniería de variables


In [91]:
# Crear variable que se llama "Grupo_Origen" tal que: 
#Si Pais es Ecuador sea "Nacional", si es Colombia, Perú sea "CAN", Caso Contrario sea "Exterior"
lista_aux = []
for i in range(0,len(df_autos)):
  if df_autos["Pais_Origen"][i] == "ECUADOR":
    lista_aux.append("Nacional")
  elif df_autos["Pais_Origen"][i] == "COLOMBIA" or df_autos["Pais_Origen"][i] == "PERU":
    lista_aux.append("CAN")
  else:
    lista_aux.append("Exterior")

df_autos["Grupo_Origen"] = lista_aux

In [92]:
#Ver un conteo de frecuencias de esta variable creada
frec_origen = df_autos["Grupo_Origen"].value_counts()
d_frec_origen = {"Grupo_Origen":frec_origen.index,"Frecuencia":frec_origen}
df_frec_origen = pd.DataFrame(d_frec_origen)
df_frec_origen.index = range(0,len(frec_origen))
df_frec_origen["Frec_Relativa"] = np.round(100*df_frec_origen.Frecuencia/np.sum(df_frec_origen.Frecuencia),2)
df_frec_origen

,Grupo_Origen,Frecuencia,Frec_Relativa
0,Exterior,224688,64.36
1,Nacional,78498,22.49
2,CAN,45923,13.15


In [ ]:
#Crear variable "Rango_Avaluo" que contenga 3 rangos de avaluo tal que:
#Si clase es Automovil, Camioneta o Jeep:
#Si Avaluo es menor a 15000 sea "Bajo", Si esta entre 15000 y 40000 sea "Medio", Caso contrario sea "Alto"
#Para cualquier otra clase el Rango es "No Aplica"

#Construir tabla de frecuencias de esta variable creada

In [111]:
#Identificar casos atipicos de avaluo en automóviles

#Tabla auxiliar filtrando solamente automóviles en Clase
df_auxiliar = df_autos[df_autos["Clase"] == "AUTOMOVIL"]
df_auxiliar.head()

#Obtener límites para identificar casos atípicos
iqr = df_auxiliar["Valor_Avaluo"].quantile(0.75) - df_auxiliar["Valor_Avaluo"].quantile(0.25)
lim_sup = df_auxiliar["Valor_Avaluo"].quantile(0.75) + 1.5*iqr
lim_inf = df_auxiliar["Valor_Avaluo"].quantile(0.25) - 1.5*iqr
print(lim_sup)
print(lim_inf)

25990.0
9990.0


In [116]:
#Crear columna que identifique casos atípicos
lista_aux = []
for i in range(0,len(df_autos)):
  if df_autos["Clase"][i] == "AUTOMOVIL":
    if df_autos["Valor_Avaluo"][i] > lim_sup or  df_autos["Valor_Avaluo"][i] < lim_inf:
      lista_aux.append("ATIPICO")
    else:
      lista_aux.append("OK")
  else:
    lista_aux.append("OK")

df_autos["Caso"] = lista_aux

In [120]:
#Contar y promedio avaluo por caso
df_autos.groupby(["Caso"], as_index = False).agg({"Cod_Vehiculo":"count","Valor_Avaluo":"mean"})

,Caso,Cod_Vehiculo,Valor_Avaluo
0,ATIPICO,4934,41960.190411
1,OK,344175,28524.951680


In [122]:
#Sustituir casos atipicos por un valor de avaluo mediano
avaluo_ref = df_auxiliar["Valor_Avaluo"].median()

for i in range(0,len(df_autos)):
  if df_autos["Caso"][i] == "ATIPICO":
    df_autos["Valor_Avaluo"][i] = avaluo_ref
  else:
    continue

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [123]:
#Contar y promedio avaluo por caso
df_autos.groupby(["Caso"], as_index = False).agg({"Cod_Vehiculo":"count","Valor_Avaluo":"mean"})

,Caso,Cod_Vehiculo,Valor_Avaluo
0,ATIPICO,4934,17490.00000
1,OK,344175,28524.95168
